In [16]:
import torch
import pyro
import pyro.infer as infer
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import math
import torch.distributions.constraints as constraints
import pyro.distributions as dist
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

print(torch.__version__)

torch.manual_seed(101)

0.4.0


In [19]:
# Implementation of stochastic variational inference

"""
Introduce a variational distribution with parameters called as variational parameters
In the context of pyro, this distribution is known as the guide

The guide serves as an approximation to the true posterior


To move the guide closer to the posterior, we need an appropriate objective function.

We use the Evidence Lower Bound (ELBO)

ELBO≡𝔼qϕ(z)[logpθ(x,z)−logqϕ(z)]



"""

def model(data):
    
    # define the parameters that control the beta prior
    alpha_0 = torch.Tensor([10.0])
    beta_0 = torch.Tensor([10.0])
    
    # sample fairness f from beta prior
    f = pyro.sample('latent_fairness', dist.Beta(alpha_0, beta_0))
    # Loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
        # Conditioned on the 'latent fairness' random variable, we observe each of the datapoints using a bernoulli likelihood

# Define a corresponding guide i.e. an appropriate variational
# distribution for the latent random variable f.


def guide(data):
    # register the 2 variational parameters with pyro
    alpha_q = pyro.param('alpha_q', torch.Tensor([15.0]), constraint=constraints.positive)
    beta_q = pyro.param('beta_q', torch.Tensor([15.0]), constraint=constraints.positive)

    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))



In [10]:
# enable validation (e.g. validate parameters of distributions)
pyro.enable_validation(True)

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(6):
    data.append(torch.Tensor([1.0]))
for _ in range(4):
    data.append(torch.Tensor([0.0]))

In [13]:
# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())


# do gradient steps
for step in range(2000):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')


.

.

.

.

.

...

.

.

.

.

.

In [14]:
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

In [17]:
# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))


based on the data and our prior belief, the fairness of the coin is 0.540 +- 0.090
